## Oligodendrocyte notebook

In [1]:
import pandas
import seaborn
import numpy
import matplotlib.pyplot
import scanpy
from sklearn.decomposition import PCA

import os
import glob
import pickle
import pandas 
import numpy 

from dask.diagnostics import ProgressBar

from arboreto.utils import load_tf_names
from arboreto.algo import grnboost2

from ctxcore.rnkdb import FeatherRankingDatabase 
from pyscenic.utils import modules_from_adjacencies, load_motifs
from pyscenic.prune import prune2df, df2regulons
from pyscenic.aucell import aucell

In [2]:
matplotlib.rcParams.update({'font.size':20, 'font.family':'sans-serif', 
                            'xtick.labelsize':30, 'ytick.labelsize':30, 'figure.figsize':(12, 6.75), 
                            'axes.labelsize':40})

In [3]:
pwd

'/Users/lidiayung/github/notebooks/Oligo'

In [4]:
cd /Users/lidiayung/github/notebooks/oligo/SCP12/expression

/Users/lidiayung/github/notebooks/Oligo/SCP12/expression


In [ ]:
#Input data from 
## https://singlecell.broadinstitute.org/single_cell/study/SCP12/oligodendroglioma-intra-tumor-heterogeneity
#! curl "https://singlecell.broadinstitute.org/single_cell/api/v1/bulk_download/generate_curl_config?accessions=SCP12&auth_code=vECFXlfo&directory=all&context=study"  -o cfg.txt; curl -K cfg.txt && rm cfg.txt

In [5]:
filename = 'OG_processed_data_portal.txt'
data1 = pandas.read_table(filename, delimiter='\t', header=0,index_col=0)
data = data1.transpose()
data.head()

GENE,A1BG,A1BG-AS1,A1CF,A2M,A2M-AS1,A2ML1,A2MP1,A4GALT,A4GNT,AA06,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
MGH36_P6_A12,0.0,0.0,0.00000,5.7056,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.53506,0.35163,0.00000,1.3618,1.599800,0.0
MGH36_P6_H09,0.0,0.0,0.00000,4.4370,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.14535,0.00000,2.21850,3.2621,0.000000,0.0
MGH53_P4_G04,0.0,0.0,0.00000,8.0276,4.5347,0.32077,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.77740,0.70752,0.84398,0.0000,0.000000,0.0
MGH36_P10_G12,0.0,0.0,0.00000,5.6288,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.11700,0.00000,1.22590,0.0000,0.269030,0.0
MGH53_P2_H12,0.0,0.0,0.02148,0.0000,0.0000,0.00000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.12168,0.22157,0.14405,4.9809,0.035624,0.0


In [6]:
print(data.index)

Index(['MGH36_P6_A12', 'MGH36_P6_H09', 'MGH53_P4_G04', 'MGH36_P10_G12',
       'MGH53_P2_H12', 'MGH53_P4_D10', 'MGH53_P4_D01', 'MGH36_P6_B07',
       'MGH36_P10_B12', 'MGH53_P2_G11',
       ...
       '93_P10_H06', '93_P8_B12', '93_P8_D09', '93_P9_D11', '93_P10_G08',
       '93_P8_H06', '93_P9_C07', '93_P8_A12', '93_P8_C01', '93_P9_F06'],
      dtype='object', length=4347)


In [7]:
data.columns

Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1',
       'A4GALT', 'A4GNT', 'AA06',
       ...
       'ZWILCH', 'ZWINT', 'ZXDA', 'ZXDB', 'ZXDC', 'ZYG11A', 'ZYG11B', 'ZYX',
       'ZZEF1', 'ZZZ3'],
      dtype='object', name='GENE', length=23686)

In [8]:
data.shape

(4347, 23686)

In [9]:
pacients_ids = [element.split('_')[0] for element in data.index.to_list()]
pacients_unique_id = list(set(pacients_ids))
print(pacients_unique_id)
#MGH36,53,54 deeper analysis

['97', 'MGH53', 'MGH97', 'MGH93', '93', 'MGH54', 'MGH36', 'MGH60']


In [10]:
test = [element.split('_')[0] for element in data.index.to_list()]

In [11]:
patients= []
for i in test:
    if 'MGH' in i:
        i = i
    else:
        i='MGH' +i
    patients.append(i)

In [12]:
pacients_uid = list(set(patients))
pacients_uid

['MGH53', 'MGH97', 'MGH93', 'MGH54', 'MGH36', 'MGH60']

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4347 entries, MGH36_P6_A12 to 93_P9_F06
Columns: 23686 entries, A1BG to ZZZ3
dtypes: float64(23686)
memory usage: 785.6+ MB


join datasets

In [ ]:
DATABASES_GLOB = os.path.join("hg38_10kbp_up_10kbp_down_full_tx_v10_clust.genes_vs_motifs.rankings.feather")
MOTIFS_HGNC_FNAME = os.path.join('motifs-v9-nr.hgnc-m0.001-o0.0.tbl')
CURATED_TFS_HGNC_FNAME = os.path.join('lambert2018.txt')

OUT_TFS_HGNC_FNAME = os.path.join('hs_hgnc_tfs.txt')
OUT_TFS_HGNC_FNAME = os.path.join('hs_hgnc_curated_tfs.txt')

In [ ]:
ADJACENCIES_FNAME = os.path.join("adjacencies.tsv")
MODULES_FNAME = os.path.join("modules.p")
MOTIFS_FNAME = os.path.join("motifs.csv")
REGULONS_FNAME = os.path.join("regulons.p")

In [ ]:
db_fnames = glob.glob(DATABASES_GLOB)
def name(fname):
    return os.path.splitext(os.path.basename(fname))[0]
dbs = [FeatherRankingDatabase(fname=fname, name=name(fname)) for fname in db_fnames]
dbs

In [ ]:
cd /Users/lidiayung/github/notebooks

In [ ]:
df_motifs_hgnc = pandas.read_csv(MOTIFS_HGNC_FNAME, sep='\t')
hs_tfs = df_motifs_hgnc.gene_name.unique()
with open(OUT_TFS_HGNC_FNAME, 'wt') as f:
    f.write('\n'.join(hs_tfs) + '\n')
len(hs_tfs)

In [ ]:
with open(CURATED_TFS_HGNC_FNAME, 'rt') as f:
    hs_curated_tfs = list(map(lambda s: s.strip(), f.readlines()))
len(hs_curated_tfs)

In [ ]:
hs_curated_tfs_with_motif = list(set(hs_tfs).intersection(hs_curated_tfs))
len(hs_curated_tfs_with_motif)

In [ ]:
with open(OUT_TFS_HGNC_FNAME, 'wt') as f:
    f.write('\n'.join(hs_curated_tfs_with_motif) + '\n')

In [ ]:
gene_names= list(data.columns)
tf_names = load_tf_names(MOTIFS_HGNC_FNAME)

In [ ]:
type(tf_names)

Phase I: Inference of co-expression modules

In [ ]:
adjancencies = grnboost2(expression_data=data,gene_names=gene_names ,tf_names=hs_curated_tfs, verbose=True)

In [ ]:
adjancencies.head()

In [ ]:
adjancencies.shape

In [ ]:
adjancencies.to_csv(ADJACENCIES_FNAME, index=False, sep='\t')
adjacencies = pandas.read_csv(ADJACENCIES_FNAME, sep='\t')

In [ ]:
modules_og = list(modules_from_adjacencies(adjacencies, data))

In [ ]:
with open(MODULES_FNAME, 'wb') as f:
    pickle.dump(modules_og, f)

In [ ]:
df = prune2df(dbs, modules_og, MOTIFS_HGNC_FNAME)
df.head()

In [ ]:
regulons = df2regulons(df)

Groups

In [ ]:
df_MGH36 = data.filter(like='MGH36',axis=0)
df_MGH36.shape
#print(df_MGH36.index.to_list())

In [ ]:
df_MGH53 = data.filter(like='MGH53',axis=0)
df_MGH53.shape

In [ ]:
df_MGH54 = data.filter(like='MGH54',axis=0)
df_MGH54.shape

In [ ]:
scanpy.pp.filter_cells(df_MGH36, min_genes=0)

In [ ]:
overall_expression = numpy.sum(df_MGH36, axis=0)
sorted = overall_expression.sort_values(ascending=False, inplace=True)
overall_expression.head(10)

In [ ]:
overall_expression.tail()

In [ ]:
#set_xlabel("log2(TPM/10 + 1)")
seaborn.displot(overall_expression, kde=True,stat="density",rug=True)
matplotlib.pyplot.show()

In [ ]:
#for column in columns_ls:
#    y = data[column].mean()
#    data[column].fillna(y,inplace=False)
#print(data.shape)

In [ ]:
data_full.shape

In [ ]:
data['A1BG']
#A2Mavg=A2mmean.mean()
#print(A2Mavg)

### t-SNE

In [ ]:
filtered_df = data[data.isnull()]
filtered_df.shape

In [ ]:
data_full.head()

In [14]:
pca= PCA(n_components=5)
pca.fit(data)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
transformed = pca.transform(data)
transformed.shape

In [ ]:
pca_data = pandas.DataFrame(transformed)
pca_data.insert(0,"Pacients_IDs",pacients_ids)

In [ ]:
colors= {'MGH97':'b','MGH60':'g',"MGH36":'r',"MGH93":'orange',"MGH54":'m',"MGH53":'y'}
for element in pacients_uid:
    x = transformed.loc[transformed["pacients"]== element][0]
    y = transformed.loc[transformed["pacients"]== element][1]
    matplotlib.pyplot.legend()
    matplotlib.pyplot.scatter(x, y,c=colors[element],label=str(element))   
matplotlib.pyplot.xlabel("PCA1")
matplotlib.pyplot.ylabel("PCA2")
matplotlib.pyplot.show()

In [ ]:
import sklearn.manifold 
tsne_op = sklearn.manifold.TSNE()
data_tsne = tsne_op.fit_transform(data)

In [ ]:
tsne_data = pandas.DataFrame(data_tsne)
tsne_data.insert(0,"patients",patients)
tsne_data.head()

In [ ]:
tsne_data.shape

In [ ]:
colors= {'MGH97':'b','MGH60':'g',"MGH36":'r',"MGH93":'orange',"MGH54":'m',"MGH53":'y'}
for element in pacients_uid:
    x = tsne_data.loc[tsne_data["patients"]== element][0]
    y = tsne_data.loc[tsne_data["patients"]== element][1]
    matplotlib.pyplot.legend()
    matplotlib.pyplot.scatter(x, y,c=colors[element],label=str(element)) 
matplotlib.pyplot.xlabel("t-SNE1")
matplotlib.pyplot.ylabel("t-SNE2")    
matplotlib.pyplot.show() 